<a href="https://colab.research.google.com/github/nneko1231/PDF-Converter/blob/main/PDF_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# will mount drive if MOUNT_DRIVE is True
import os
import subprocess
import shutil
from google.colab import drive, output

# Add a parameter to control mounting Google Drive
MOUNT_DRIVE = False  #@param {type:"boolean"}

# mount drive if MOUNT_DRIVE is True
if MOUNT_DRIVE:
    drive.mount('/content/drive')

In [ ]:
#@title Install Libary
!apt-get install -y poppler-utils
!pip install pdf2image

In [ ]:
#@title Run
from pdf2image import convert_from_path
from PIL import Image
import os
import zipfile
from tqdm import tqdm
import multiprocessing

PDF_PATH = "" #@param {type:"string"}
IMAGE_FORMAT = "png" # @param ["png","jpg"]
IMAGE_DPI = 300 #@param ["72","150","300","600"]
IMAGE_NAME = "" #@param {type:"string"}
IMAGE_FORMAT_NUMBER = 0 #@param {"type":"integer"}
IMAGE_PATH = "" #@param {type:"string"}
ZIP_OUTPUT_PATH = "" #@param {type:"string"}

os.makedirs(IMAGE_PATH, exist_ok=True)

max_threads = multiprocessing.cpu_count()

pages = convert_from_path(
    PDF_PATH,
    dpi=int(IMAGE_DPI),
    thread_count=max_threads
)

image_files = []
print("📄 Converting PDF to images...")
for i, page in enumerate(
    tqdm(pages, desc="Converting", unit="page", total=len(pages)),
    start=IMAGE_FORMAT_NUMBER
):
    file_name = f"{IMAGE_NAME}_{str(i).zfill(2)}.{IMAGE_FORMAT}"
    file_path = os.path.join(IMAGE_PATH, file_name)
    page.save(file_path, IMAGE_FORMAT.upper())
    image_files.append(file_path)

print("\n📦 Creating ZIP file...")
zip_filename = f"{ZIP_OUTPUT_PATH}.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in tqdm(image_files, desc="Zipping", unit="file"):
        zipf.write(file, os.path.basename(file))

print(f"\n✅ All images saved in: {IMAGE_PATH}")
print(f"✅ ZIP file created at: {zip_filename}")

In [ ]:
#@title Zip Download
from google.colab import files

if not ZIP_OUTPUT_PATH.endswith(".zip"):
    zip_file = ZIP_OUTPUT_PATH + ".zip"
else:
    zip_file = ZIP_OUTPUT_PATH

files.download(zip_file)
